In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
DATASET_PATH = '/kaggle/input/tensorflow-great-barrier-reef/train_images/'
CKPT_PATH = '/kaggle/input/cots-nemo-team/f0-10.pt'
CKPT_PATH1 = '/kaggle/input/reef-inference/f2_sub2.pt'

IMG_SIZE  = 2400
CONF      = 0.40
IOU       = 0.95

IMG_SIZE1  = 6400
CONF1      = 0.3
IOU1       = 0.95

import numpy as np
import pandas as pd
import sys
import cv2
import torch
from PIL import Image

sys.path.append('../input/tensorflow-great-barrier-reef')

In [ ]:
%cd /kaggle/input/augs-tweaked/albumentations
%pip install -U .
%cd /kaggle/working

In [ ]:
import torchvision

In [ ]:
!pip install ../input/reef-inference/ensemble_boxes-1.0.7-py3-none-any.whl

In [ ]:
from ensemble_boxes import *

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp ../input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index

In [ ]:
%cd /kaggle/working

In [ ]:
def RecoverCLAHE(sceneRadiance):
    clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(4, 4))
    for i in range(3):
        sceneRadiance[:, :, i] = clahe.apply((sceneRadiance[:, :, i]))

    return sceneRadiance

In [ ]:
def load_model(ckpt_path, conf=0.15, iou=0.30):
    model = torch.hub.load('/kaggle/input/yolo-lib',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload = True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 100  # maximum number of detections per image
    return model


In [ ]:
def predict(models, img, size = 1280, augment = False):
    bboxes = []
    scores = []
    box_list = []
    score_list = []
    label_list = []
    for i, model in enumerate(models):
        if i==0:
            results = model(img, size=6400, augment = False)  # custom inference size
        elif i==1:
            results = model(img, size=4800, augment = False)
        elif i==2:
            results = model(img, size=3200, augment = False)
        else:
            results = model(img, size=2400, augment = True)
        preds   = results.pandas().xyxy[0].values
        preds[:,[0,2]]=preds[:,[0,2]]/1280
        preds[:,[1,3]]=preds[:,[1,3]]/720
        bboxes, scores, labels = weighted_boxes_fusion([preds[:,:4]], [preds[:,4]], [preds[:,5]], weights=None, iou_thr=0.25, skip_box_thr=0.001)
        box_list.append(bboxes)
        score_list.append(scores)
        label_list.append(labels)
    

    bboxes, scores, labels = weighted_boxes_fusion(box_list, score_list, label_list, weights=None, iou_thr=0.25, skip_box_thr=0.001)
    bboxes[:,[0,2]]=bboxes[:,[0,2]]*1280
    bboxes[:,[1,3]]=bboxes[:,[1,3]]*720
    bboxes1, scores1, labels1 = [],[], []
    for b,s,l in zip(bboxes,scores,labels):
        if s >= 0.23:
            bboxes1.append(b)
            scores1.append(s)
            labels1.append(l)


    return bboxes1, scores1

In [ ]:
from norfair import Detection, Tracker

def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        ratio = w/h
        if ratio > 0.45 and ratio < 2.22:
            result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result


def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

In [ ]:
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=25,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

frame_id = 0

In [ ]:
def show_prediction(img, bboxes):
    colors = [(0, 0, 255)]

    obj_names = ["s"]

    for box in bboxes:
        cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255,0,0), 2)
    
    img = Image.fromarray(img).resize((800, 400))
    return img

In [ ]:
model1 = load_model(CKPT_PATH1,CONF1,IOU1)
model2 = load_model(CKPT_PATH1,0.4,IOU1)
model3 = load_model(CKPT_PATH1,0.4,IOU1)
model4 = load_model(CKPT_PATH,0.4,IOU)
models = [model1,model2,model3,model4]

In [ ]:
dir = f'{DATASET_PATH}'
imgs = [dir + f for f in ('video_0/9674.jpg',
                          'video_0/20.jpg', 
                          'video_0/17.jpg', 
                          'video_0/100.jpg',
                          'video_0/246.jpg',
                          'video_0/337.jpg',
                          'video_0/358.jpg',
                          'video_0/1566.jpg',
                          'video_0/1854.jpg',
                          'video_0/1884.jpg',
                          'video_0/4224.jpg',
                          'video_0/4540.jpg',
                          'video_0/4582.jpg',
                          'video_0/4664.jpg',
                          'video_0/8215.jpg',
                          'video_0/8897.jpg',
                          'video_0/8928.jpg',
                          'video_0/9441.jpg',
                          'video_0/9859.jpg',
                          'video_0/12267.jpg',
                          'video_2/5748.jpg',
                          'video_2/5772.jpg',
                          'video_2/5820.jpg',
                          'video_2/5409.jpg',
                          'video_2/5482.jpg',
                          'video_2/5679.jpg',
                          'video_2/5712.jpg',
                          'video_2/5730.jpg',
                          'video_2/5751.jpg',
                          'video_2/5817.jpg',
                          'video_2/5868.jpg',
                          'video_2/6254.jpg',
                          'video_2/6339.jpg',
                          'video_2/10622.jpg',
                          'video_1/4159.jpg', 
                          'video_1/4183.jpg', 
                          'video_1/4501.jpg',
                          'video_1/5474.jpg',
                          'video_1/625.jpg',
                          'video_1/616.jpg',
                          'video_1/672.jpg',
                          'video_1/684.jpg',
                          'video_1/850.jpg',
                          'video_1/1927.jpg',
                          'video_1/2000.jpg',
                          'video_1/3903.jpg',
                          'video_1/3945.jpg',
                          'video_1/4051.jpg',
                          'video_1/4078.jpg',
                          'video_1/4096.jpg',
                          'video_1/4126.jpg',
                          'video_1/4456.jpg',
                          'video_1/4525.jpg',
                          'video_1/5267.jpg',
                          'video_1/5366.jpg',
                          'video_1/5411.jpg',
                          'video_1/5429.jpg',
                          'video_1/5492.jpg',
                          'video_1/5661.jpg',
                          'video_1/5892.jpg',
                          'video_1/6747.jpg',
                          'video_1/9082.jpg',)]

for img in(imgs):
    im = cv2.imread(img)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    bboxes, scores = predict(models, im, size = IMG_SIZE, augment = True)
    display(show_prediction(im, bboxes))

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
#model = load_model(CKPT_PATH, conf=CONF, iou=IOU)

for (image_np, sample_prediction_df) in iter_test:
    
    bboxes, scores = predict(models, image_np, size = IMG_SIZE, augment = True)
    
    predictions = []
    detects = []
    
    for i in range(len(bboxes)):
        box = bboxes[i]
        score = scores[i]
        x_min = int(box[0])
        y_min = int(box[1])
        x_max = int(box[2])
        y_max = int(box[3])
        
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min
        detects.append([x_min, y_min, x_max, y_max, score])
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        if last_detected_frame_id <= frame_id-2:  continue
            
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]
        ratio = bbox_width/bbox_height
        if ratio > 0.45 and ratio < 2.22:
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)
    frame_id += 1

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()